In [1]:
example = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [2]:
with open('./data/Day 19/input.txt') as inputFile:
    data = inputFile.read()

In [3]:
import re

In [4]:
def parse_input(inputstr):
    rules = {}
    parts = []
    rulesection = True
    for line in inputstr.splitlines():
        if line == "":
            rulesection = False
            continue
        if rulesection:
            identifier, rule = line.strip("}").split("{")
            rules[identifier] = re.split(r":|,", rule)
        else:
            cleaned_line = line.strip('{}')
            parts.append(
                {i.split("=")[0]: int(i.split("=")[1]) for i in cleaned_line.split(",")}
            )
    return rules, parts

In [5]:
def test(rules, i, x, m, a, s):
    if eval(rules[i]):
        return rules[i+1], i+1
    else:
        return rules[i+2], i+2

In [6]:
def run_part(ruleset, part):
    current_key = 'in'
    next_action, index = test(ruleset['in'], 0, **part)
    while next_action not in ['A', 'R']:
        if next_action in ruleset.keys():
            current_key = next_action
            next_action, index = test(ruleset[next_action], 0, **part)
        else:
            next_action, index = test(ruleset[current_key], index, **part)
    return next_action

# Part 1

In [7]:
example_rules, example_parts = parse_input(example)
example_score = 0
for part in example_parts:
    part['result'] = run_part(example_rules, part)
    if part['result'] == 'A':
        example_score += part['x'] + part['m'] + part['a'] + part['s']
example_score

19114

In [8]:
rules, parts = parse_input(data)
score = 0
for part in parts:
    part["result"] = run_part(rules, part)
    if part['result'] == 'A':
        score += part['x'] + part['m'] + part['a'] + part['s']
score

325952

# Part 2

In [9]:
def get_condition_tree(rule, i):
    tree = {'condition': rule[i], True: rule[i + 1], False: rule[i + 2]}
    return tree, i

In [10]:
def create_tree(rule, rules):
    if isinstance(rule[1], list):
        rule[1] = create_tree(rule[1], rules)
    if isinstance(rule[2], list):
        rule[2] = create_tree(rule[2], rules)
    if rule[1] not in ['A', 'R'] and not isinstance(rule[1], list):
        rule[1] = create_tree(rules[rule[1]], rules)
    if rule[2] not in ['A', 'R'] and not isinstance(rule[2], list):
        rule[2] = create_tree(rules[rule[2]], rules)
    return rule

In [11]:
def cut_list(input_list):
    if len(input_list) > 3:
        input_list = input_list[:2] + [cut_list(input_list[2:])]
    return input_list

In [12]:
example_rules, _ = parse_input(example)
example_rules_2 = {}
for key, value in example_rules.items():
    value = cut_list(value)
    example_rules_2[key] = value
example_tree = create_tree(example_rules_2["in"], example_rules_2)

In [61]:
def get_valid_ranges(tree, valids):
    if isinstance(tree[1], list):
        valids = get_valid_ranges(tree[1], valids)
    if tree[1] == 'A':
        if '<' in tree[0]:
            char, value = tree[0].split('<')
            valids[char].append(('<',int(value)))
        if '>' in tree[0]:
            char, value = tree[0].split(">")
            valids[char].append(('>',int(value)))
    if isinstance(tree[2], list):
        valids = get_valid_ranges(tree[2], valids)
    if tree[2] == 'A':
        if '<' in tree[0]:
            char, value = tree[0].split('<')
            valids[char].append(('>',int(value)))
        if '>' in tree[0]:
            char, value = tree[0].split(">")
            valids[char].append(('<',int(value)))
    return valids

In [62]:
valids = {"x": [], "m": [], "a": [], "s": []}
example_valid_ranges = get_valid_ranges(example_tree, valids)
for key in example_valid_ranges.keys():
    example_valid_ranges[key] = sorted(example_valid_ranges[key], key=lambda x: x[1])
example_valid_ranges

{'x': [('<', 1416), ('<', 2440), ('>', 2662)],
 'm': [('>', 838), ('>', 1548), ('<', 1548), ('>', 2090)],
 'a': [('<', 1716)],
 's': [('>', 3448)]}

In [63]:
example_tree

['s<1351',
 ['a<2006',
  ['x<1416', 'A', ['x>2662', 'A', 'R']],
  ['m>2090', 'A', ['s<537', ['a>3333', 'R', 'R'], ['x>2440', 'R', 'A']]]],
 ['s>2770',
  ['s>3448', 'A', ['m>1548', 'A', 'A']],
  ['m<1801', ['m>838', 'A', ['a>1716', 'R', 'A']], 'R']]]

In [49]:
sum(range(0, 1315)) + sum(range(0, 2006)) + sum(range(0, 1416)) + sum(range(4000)) + \
sum(range(0, 1315)) + sum(range(0, 2006)) + sum(range(2662, 4000)) + sum(range(4000))

27203969

In [44]:
167409079868000 - (sum(range(0, 1416)) + sum(range(2440,4000)) + sum(range(838, 4000)) + sum(range(1716, 4000)) + sum(range(3448, 4000)))

167409057614561

In [29]:
rules_2 = {}
for key, value in rules.items():
    value = cut_list(value)
    rules_2[key] = value
create_tree(rules_2["in"], rules_2)

['m>2048',
 ['m>2886',
  ['s>2599',
   ['x>2056',
    ['m>3553',
     ['m>3798',
      ['a>2466', 'A', 'R'],
      ['a<1797',
       ['x>2896', 'R', ['m>3700', 'R', 'R']],
       ['s>3136', 'A', ['s<2954', 'R', ['m<3637', 'A', 'R']]]]],
     ['s<3379',
      ['a>1665',
       'R',
       ['a<877', ['m<3304', 'A', ['m<3428', 'A', ['a<346', 'R', 'A']]], 'A']],
      ['x<3059',
       ['s<3764', 'A', ['x>2644', 'A', 'A']],
       ['a<1420',
        ['m>3120', 'A', 'R'],
        ['m>3268', 'R', ['m<3061', 'R', ['x>3496', 'R', 'A']]]]]]],
    ['m<3484',
     ['x>1336',
      ['m<3123', 'A', ['x<1729', 'A', ['a<2135', 'A', 'R']]],
      ['m>3270', ['a>1950', 'A', 'A'], ['m>3040', 'R', ['x>718', 'R', 'A']]]],
     ['m>3762',
      ['m>3874', 'R', ['m<3814', 'R', 'R']],
      ['a<2631',
       ['x<1323', 'A', 'R'],
       ['m<3615',
        'A',
        ['x<922',
         ['m>3707', 'R', ['x<572', 'A', ['x<700', 'R', 'A']]],
         ['x<1528', 'R', ['m>3709', 'A', 'A']]]]]]]],
   ['s<1700',
 